In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Embedding, Dropout, Input, Concatenate
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.models import load_model
from google.colab import drive

In [2]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
second_model = load_model('/content/drive/MyDrive/Election Project/second_model.h5')
df_2013 = pd.read_csv('/content/drive/MyDrive/Election Project/Complete 2013.csv')

In [4]:
df_2013.isnull().sum()

election_date           0
election_type           0
contest_status          0
constituency_number     0
constituency_name       0
province                0
assembly                0
voter_reg              24
validated_votes         0
votes_disq              0
candidateID             0
candidate_name          0
candidate_party         0
candidate_votes         0
candidate_share         0
candidate_rank          0
outcome                 0
dtype: int64

In [5]:
df_2013['voter_reg'].fillna(df_2013['voter_reg'].mean(), inplace=True)

In [6]:
'''df_2013 = df_2013.dropna(subset=['constituency_number'])
exclude_prefixes = ['PP', 'PB', 'PK', 'PS']
df_2013 = df_2013[~df_2013['constituency_number'].str.startswith(tuple(exclude_prefixes))]'''

"df_2013 = df_2013.dropna(subset=['constituency_number'])\nexclude_prefixes = ['PP', 'PB', 'PK', 'PS']\ndf_2013 = df_2013[~df_2013['constituency_number'].str.startswith(tuple(exclude_prefixes))]"

In [7]:
df_2013.isnull().sum()

election_date          0
election_type          0
contest_status         0
constituency_number    0
constituency_name      0
province               0
assembly               0
voter_reg              0
validated_votes        0
votes_disq             0
candidateID            0
candidate_name         0
candidate_party        0
candidate_votes        0
candidate_share        0
candidate_rank         0
outcome                0
dtype: int64

In [8]:
x=df_2013['constituency_number'].unique()
len(df_2013['constituency_number'].unique())

271

In [9]:
x

array(['NA-1', 'NA-10', 'NA-100', 'NA-101', 'NA-102', 'NA-103', 'NA-104',
       'NA-105', 'NA-106', 'NA-107', 'NA-108', 'NA-109', 'NA-11',
       'NA-110', 'NA-111', 'NA-112', 'NA-113', 'NA-114', 'NA-115',
       'NA-116', 'NA-117', 'NA-118', 'NA-119', 'NA-12', 'NA-120',
       'NA-121', 'NA-122', 'NA-123', 'NA-124', 'NA-125', 'NA-126',
       'NA-127', 'NA-128', 'NA-129', 'NA-13', 'NA-130', 'NA-131',
       'NA-132', 'NA-133', 'NA-134', 'NA-135', 'NA-136', 'NA-137',
       'NA-138', 'NA-139', 'NA-14', 'NA-140', 'NA-141', 'NA-142',
       'NA-143', 'NA-144', 'NA-145', 'NA-146', 'NA-147', 'NA-148',
       'NA-149', 'NA-15', 'NA-150', 'NA-151', 'NA-152', 'NA-153',
       'NA-154', 'NA-155', 'NA-156', 'NA-157', 'NA-158', 'NA-159',
       'NA-16', 'NA-160', 'NA-161', 'NA-162', 'NA-163', 'NA-164',
       'NA-165', 'NA-166', 'NA-167', 'NA-168', 'NA-169', 'NA-17',
       'NA-170', 'NA-171', 'NA-172', 'NA-173', 'NA-174', 'NA-175',
       'NA-176', 'NA-177', 'NA-178', 'NA-179', 'NA-18', 'NA-18

In [10]:
desire_col = ['constituency_number', 'province', 'candidate_name','candidate_party','candidate_votes','candidate_share','outcome']
data = df_2013[desire_col]

In [11]:
max_candidate_name_length = 40
max_candidate_party_length = 40

In [12]:
def tokenize_and_pad(data,max_len):
  sequences = pad_sequences(data, maxlen=max_len, padding='post')
  print(sequences)
  return sequences.tolist()

In [13]:
data['candidate_name'] = data.groupby('constituency_number')['candidate_name'].transform(lambda x: x.astype(str).tolist())
data['candidate_party'] = data.groupby('constituency_number')['candidate_party'].transform(lambda x: x.astype(str).tolist())

# Extract unique constituencies
constituencies = data['constituency_number'].unique()

<ipython-input-13-df51f43c78f4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['candidate_name'] = data.groupby('constituency_number')['candidate_name'].transform(lambda x: x.astype(str).tolist())
<ipython-input-13-df51f43c78f4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['candidate_party'] = data.groupby('constituency_number')['candidate_party'].transform(lambda x: x.astype(str).tolist())


In [14]:
all_features, all_labels = [], []

for constituency_number in constituencies:
    group = data[data['constituency_number'] == constituency_number]

    features = group[['province', 'candidate_name', 'candidate_party', 'candidate_votes', 'candidate_share']]
    labels = group['outcome'].astype('category').cat.codes

    all_features.append(features)
    all_labels.append(labels)

processed_data = pd.concat(all_features, ignore_index=True)
processed_data['outcome'] = pd.concat(all_labels, ignore_index=True)

In [15]:
# Convert text columns to sequences
tok = Tokenizer(num_words=10000)
tok.fit_on_texts(processed_data['candidate_name'].astype(str))
processed_data['candidate_name_padded'] = tok.texts_to_sequences(processed_data['candidate_name'].astype(str))

tok.fit_on_texts(processed_data['candidate_party'].astype(str))
processed_data['candidate_party_padded'] = tok.texts_to_sequences(processed_data['candidate_party'].astype(str))

In [16]:
X_train, X_test, y_train, y_test = train_test_split(processed_data[['province', 'candidate_name_padded', 'candidate_party_padded', 'candidate_votes', 'candidate_share']], processed_data['outcome'], test_size=0.2, random_state=42)

In [17]:
from keras.layers import Bidirectional, Reshape

model3 = Sequential()
model3.add(second_model)
model3.add(Reshape((-1, 1)))
model3.add(LSTM(64, return_sequences=True))
model3.add(Dropout(0.2))
model3.add(LSTM(32, return_sequences=True))
model3.add(LSTM(32))
model3.add(Dense(16, activation='relu'))
model3.add(Dense(1, activation='sigmoid'))
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
X_train_padded = pad_sequences(X_train['candidate_name_padded'].tolist(), maxlen=max_candidate_name_length, padding='post', truncating='post', dtype='float32')

# Assuming max_candidate_name_length is the length of your padded sequences
timesteps = max_candidate_name_length
features = 1  # Assuming each element in the sequence is a single feature

X_train_padded = X_train_padded.reshape((X_train_padded.shape[0], timesteps, features))

# Train the model
model3.fit(X_train_padded, y_train, epochs=10, validation_split=0.2)
#model3.save('/content/drive/MyDrive/Election Project/third_model.h5')
model3.summary()

Epoch 1/10
95/95 [==============================] - 23s 93ms/step - loss: 0.4813 - accuracy: 0.9286 - val_loss: 0.2645 - val_accuracy: 0.9273
Epoch 2/10
95/95 [==============================] - 6s 61ms/step - loss: 0.2309 - accuracy: 0.9385 - val_loss: 0.2612 - val_accuracy: 0.9273
Epoch 3/10
95/95 [==============================] - 5s 57ms/step - loss: 0.2323 - accuracy: 0.9385 - val_loss: 0.2605 - val_accuracy: 0.9273
Epoch 4/10
95/95 [==============================] - 5s 53ms/step - loss: 0.2327 - accuracy: 0.9385 - val_loss: 0.2700 - val_accuracy: 0.9273
Epoch 5/10
95/95 [==============================] - 7s 71ms/step - loss: 0.2328 - accuracy: 0.9385 - val_loss: 0.2616 - val_accuracy: 0.9273
Epoch 6/10
95/95 [==============================] - 6s 58ms/step - loss: 0.2336 - accuracy: 0.9385 - val_loss: 0.2605 - val_accuracy: 0.9273
Epoch 7/10
95/95 [==============================] - 6s 65ms/step - loss: 0.2320 - accuracy: 0.9385 - val_loss: 0.2624 - val_accuracy: 0.9273
Epoch 8/10
9

In [19]:
X_test_padded = pad_sequences(X_test['candidate_name_padded'].tolist(), maxlen=max_candidate_name_length, padding='post', truncating='post', dtype='float32')
X_test_padded = X_test_padded.reshape((X_test_padded.shape[0], timesteps, features))

# Evaluate the model on the test set
evaluation = model3.evaluate(X_test_padded, y_test)
print("Test Loss:", evaluation[0])
print("Test Accuracy:", evaluation[1])

30/30 [==============================] - 1s 17ms/step - loss: 0.1923 - accuracy: 0.9524
Test Loss: 0.19232876598834991
Test Accuracy: 0.9524312615394592


In [20]:
predictions_2013 = model3.predict(X_test_padded)
predictions_2013

30/30 [==============================] - 4s 13ms/step


array([[0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788416],
       [0.05788416],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.05788417],
       [0.057